In [ ]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf, SparkContext

from typing import Tuple, List

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
invitations = sc.textFile('invitations.txt')
meetings = sc.textFile('meetings.txt')
users = sc.textFile('users.txt')
participants = sc.textFile('participations.txt')

In [ ]:
################################################################à
# Part 1
################################################################à

In [ ]:
# considering the invitations, compute the number of potential participation for each MID
# key = MID
# value = +1
# and use a reduceByKey to sum all the values
# cache the RDD, since it will be used in part 2

def filter_potential_participants(s: str) -> bool:
    fields = s.split(',')
    return fields[2] in {'Yes', 'Unknown'}

potential_participants = invitations \
                .filter(filter_potential_participants) \
                .map(lambda s: (s.split(',')[0], 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
# Map meetings to an RDD with
# key = MID
# value = maxParticipants, OrganizerUID

def parse_meetings(s: str) -> Tuple[str, Tuple[int, str]]:
    fields = s.split(',')
    mid = fields[0]
    max_part = int(fields[5])
    organizer_uid = fields[4]
    return (mid, (max_part, organizer_uid))

max_part_per_meeting = meetings \
                        .map(parse_meetings) 

In [ ]:
# join the two RDDs
# For each meeting the result contains its organizer, max. allowed participants, and number of potential participants
max_part_vs_expected = max_part_per_meeting \
                        .join(potential_participants) \
                        .cache()

In [ ]:
# Compare the number of expected and max participants for each meeting
# and use "counter" to compute the number of meetings for which
# the expected participants > the max participants
# key = MID
# value = (organizerUID , countExpectedHigherThanMax)


def expectedMoreThanAllowed(v: Tuple[str, Tuple[Tuple[int, str], int]]) -> bool:
    max_part_info, expected_info = v[1]
    max_part, organizer_uid = max_part_info
    if expected_info > max_part:
        return True
    else:
        return False
    

def transform_expected_vs_actual(v: Tuple[str, Tuple[Tuple[int, str], int]]) -> Tuple[str, int]:
    max_part_info, expected_info = v[1]
    max_part, organizer_uid = max_part_info
    return (organizer_uid, 1)


expected_greaterThanMaxPart = max_part_vs_expected\
                        .filter(expectedMoreThanAllowed)\
                        .map(transform_expected_vs_actual)

In [ ]:
# sum the two counters for each userId
# key = UID
# value = countExpectedHigherThanMax
exceed_numoccurrences = expected_greaterThanMaxPart \
                .reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
# select those entries for which the number of meetings is greater than 15 
res1 = exceed_numoccurrences.filter(lambda v: v[1] > 15) \
                    .keys()

In [ ]:
res1.saveAsTextFile('result1.txt')

In [ ]:
################################################################à
# Part 2
################################################################à

In [ ]:
# start from participations and determine for each meeting the distinct users who participated
# key = MID
# value = UID
#
# then, set values to 1 and sum everything to compute the number of actual participants
# key = MID
# value = number of distinct participants
# and filter only those meetings with at least 2 distinct participants

def parse_participants(s: str) -> Tuple[str, str]:
    fields = s.split(',')
    mid = fields[0]
    uid = fields[1]
    return mid, uid

meetings_with_at_least_2part = participants \
                                .map(parse_participants) \
                                .distinct() \
                                .mapValues(lambda v: 1) \
                                .reduceByKey(lambda v1, v2: v1 + v2) \
                                .filter(lambda v: v[1] >= 2)

In [ ]:
# Select the meetings with many potential participants (>10)
meetings_with_many_potential_part = max_part_vs_expected\
                                        .filter(lambda v: v[1][1] > 10)

In [ ]:
# Among the meetings with many potential participants remove those with a number of actual participants >=2.
# These are the meetings satifying both constraints.

meetingManyPotParts_FewActParts = meetings_with_many_potential_part.subtractByKey(meetings_with_at_least_2part)

In [ ]:
# Map to (Key=OrgUID, value=+1)
# Use reduceByKey to count the number of meeetings satisfying the constraints for each organizing user

# Input (MID, ((MaxParts, OrgUID), #PotParticipants) )
def mapOrgUIDOne(pair: Tuple[str, Tuple[Tuple[int, str], int]]) -> Tuple[str, int]:
    orgUID = pair[1][0][1]
    
    return (orgUID, 1)

res2 = meetingManyPotParts_FewActParts\
        .map(mapOrgUIDOne)\
        .reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
res2.saveAsTextFile('result2.txt')